In [ ]:
# Question 5:
import numpy as np
import matplotlib.pyplot as plt

k_values = np.arange(0, 10.1, 0.1)
x_values = 10 ** (-k_values)

y_values = (1 - np.cos(x_values)) / (x_values ** 2)

error_values = np.abs(y_values - 0.5)

plt.figure(figsize=(8, 5))
plt.plot(k_values, error_values, label=r'$|y - 0.5|$', color='b')
plt.yscale('log')  
plt.xlabel(r'$k$', fontsize=12)
plt.ylabel(r'$|y - 0.5|$', fontsize=12)
plt.title(r'Plot of $|y - 0.5|$ vs. $k$', fontsize=14)
plt.axvline(x=4, color='r', linestyle='--', label='k=4')
plt.axvline(x=8, color='g', linestyle='--', label='k=8')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Question 6:
import numpy as np

def bisect_iterations(g, a, b, n):
    for i in range(n):
        m = a + (b-a)/2
        if np.sign(g(a)) == np.sign(g(m)):
            a = m
        else:
            b = m
    return m

def fixed_point_iteration(f, x_0, N, tol = 0):
    n = 0
    while np.abs(f(x_0) - x_0) > tol and n < N:
        x_0 = f(x_0)
        n += 1
    return x_0, n

def newton_method(x_0, f, f_prime, n):
    for i in range(n):
        x_0 = x_0 - (f(x_0)/f_prime(x_0))
    return x_0

def func(x):
    return x**2 - 2

def func_prime(x):
    return 2*x

def g(x):
    return (x/2) + (1/x)

def print_res(n):
    print("### {0} ###".format(n))
    val_b = bisect_iterations(func, 0.5, 5, n)
    val_f = fixed_point_iteration(g, 5, n)
    val_n = newton_method(5, func, func_prime, n)
    print("(root) b, f, n: {0}, {1}, {2}".format(val_b, val_f[0], val_n))
    print("(abs_err) b, f, n: {0}, {1}, {2}".format(np.abs(val_b - np.sqrt(2)), np.abs(val_f[0] - np.sqrt(2)), np.abs(val_n - np.sqrt(2))))


print_res(1)
print_res(2)
print_res(3)
print_res(4)
print_res(5)

In [ ]:
# Question 7:
import numpy as np
import matplotlib.pyplot as plt

class FloatingPointSystem():
    def __init__(self, beta, p, L, U):
        self.beta = beta
        self.p = p
        self.L = L
        self.U = U

    def __get_exponent(self, x):
        i = self.L
        while self.beta**(i + 1) < abs(x) and i < self.U:
            i = i + 1
        return i
    
    def __compute_fl(self, m, e):
        fl = 0
        for i in range(self.p):
            fl += m[i] * self.beta**(e - i)
        return fl

    def __get_mantissa_ch(self, x, e):
        M = np.zeros(self.p, dtype=int)
        i = 0
        x = abs(x)

        # ensure first digit is at least 1
        M[0] = 1
        x -= self.beta**e

        while x > 0 and i < self.p:
            if x - self.beta**e >= 0 and M[i] < self.beta - 1:
                M[i] = M[i] + 1
                x -= self.beta**e
            else:
                e -= 1
                i += 1
        return M

    def fl_ch(self, x: float):
        E = self.__get_exponent(x)
        M = self.__get_mantissa_ch(x, E)
        return self.__compute_fl(M, E)
    
    def fl_rn(self, x):
        e = self.__get_exponent(x)
        M_low = self.__get_mantissa_ch(x, e)
        fl_low = self.__compute_fl(M_low, e)
        m_numbers = self.get_machine_numbers()
        higher_m_numbers = [n for n in m_numbers if n > fl_low]
        if len(higher_m_numbers) == 0:
            fl_high = fl_low
        else:
            fl_high = min(higher_m_numbers)

        if abs(x - fl_low) > abs(x - fl_high):
            return fl_high
        else:
            return fl_low
    
    def __get_m_numbers_for_exponent(self, E, pos):
        '''Run with initial values E and 0.''' 
        m_numbers = []
        if E < self.L - self.p + 1 or pos > self.p - 1:
            return [0]
        else:
            for i in range(1, self.beta):
                 m_numbers.extend([i * self.beta**(E) + x for x in self.__get_m_numbers_for_exponent(E-1, pos + 1)])
            if pos > 0:
                m_numbers.extend(self.__get_m_numbers_for_exponent(E-1, pos + 1))
            return m_numbers

    def get_machine_numbers(self):
        m_numbers = []
        for E in range(self.L, self.U + 1):
            m_numbers.extend(self.__get_m_numbers_for_exponent(E, 0))
        m_numbers = set(m_numbers)
        return m_numbers
    
    def get_min(self):
        return self.beta**self.L

    def get_max(self):
        M = np.ones(self.p) * (self.beta - 1)
        return self.__compute_fl(M, self.U)
     
class Plotter:
    def __init__():
        super()

    @staticmethod
    def get_rel_error(x_hat, x):
        return np.divide(np.abs(x_hat - x), np.abs(x))
    
    @staticmethod
    def get_abs_error(x_hat, x):
        return np.abs(x_hat - x)

    @staticmethod
    def plot(fl_sys: FloatingPointSystem, range: np.ndarray, name):
        # Plot 1:
        N = 400
        x_axis = np.linspace(range[0], range[1], 400)
        vec_fl_ch = np.vectorize(fl_sys.fl_ch)
        vec_fl_rn = np.vectorize(fl_sys.fl_rn)
        fl_ch = vec_fl_ch(x_axis)
        fl_rn = vec_fl_rn(x_axis)
        plt.figure(figsize=(8,5))
        plt.plot(x_axis, fl_ch, color='b', linestyle='dotted')
        plt.plot(x_axis, fl_rn, color='r')
        machine_numbers = np.array(list(fl_sys.get_machine_numbers()))
        plt.scatter(machine_numbers, machine_numbers, color='r', marker='o')
        plt.axvline(fl_sys.get_min(), color='black', label="min")
        plt.axvline(fl_sys.get_max(), color='black', label="max")
        plt.axvspan(range[0], fl_sys.get_min(), color='red', alpha = 0.4, label='Underflow')
        plt.axvspan(fl_sys.get_max(), range[1], color='red', alpha = 0.4, label='Overflow')
        plt.xlabel("x")
        plt.ylabel("Floating-point approximation of x")
        plt.title("Floating point representations for {0} on (0,7)".format(name))
        loc = "lower right"
        plt.legend(["Chopping", "Rounding to nearest", "Machine numbers", "Min machine number", "Max machine number", "Underflow/overflow"], loc=loc)
        plt.savefig("figures/{0}_1.png".format(name))

        abs_error_ch = Plotter.get_abs_error(fl_ch, x_axis)
        abs_error_rn = Plotter.get_abs_error(fl_rn, x_axis)
        plt.figure(figsize=(8,5))
        plt.plot(x_axis, abs_error_ch, color='b', linestyle='dotted')
        plt.plot(x_axis, abs_error_rn, color='r')
        plt.xlabel("x")
        plt.ylabel("Absolute error")
        plt.title("Absolute error graph for {0}".format(name))
        plt.legend(["Chopping", "Rounding to nearest"], loc="upper center")
        plt.savefig("figures/{0}_2.png".format(name))

        rel_error_ch = Plotter.get_rel_error(fl_ch, x_axis)
        rel_error_rn = Plotter.get_rel_error(fl_rn, x_axis)
        plt.figure(figsize=(8,5))
        plt.plot(x_axis, rel_error_ch, color='b', linestyle='dotted')
        plt.plot(x_axis, rel_error_rn, color='r')
        plt.axhline(fl_sys.beta**(fl_sys.p - 1), color = 'black', linestyle='dotted')
        plt.axhline(0.5 * fl_sys.beta**(fl_sys.p - 1), color = 'black')
        plt.xlabel("x")
        plt.ylabel("Relative error")
        plt.title("Relative error graph for {0}".format(name))
        plt.legend(["Chopping", "Rounding to nearest", "Max relative error chopping", "Max relative error rounding nearest"], loc="upper center")
        plt.savefig("figures/{0}_3.png".format(name))

Plotter.plot(FloatingPointSystem(2, 2, -1, 1), [0, 7], "F(2, 2, -1, 1)")
Plotter.plot(FloatingPointSystem(2, 4, -1, 1), [0, 7], "F(2, 4, -1, 1)")
Plotter.plot(FloatingPointSystem(2, 2, -2, 2), [0, 7], "F(2, 2, -2, 2)")

In [ ]:
# Question 8:
import numpy as np

def func_of_i(i):
    return (1500/i) * ((1 + i)**240 - 1) - 750000

def bisect_tolerance(g, a, b, tol):
    while ((b-a) > tol):
        m = a + (b-a)/2
        if np.sign(g(a)) == np.sign(g(m)):
            a = m
        else:
            b = m
    return m

ans = bisect_tolerance(func_of_i, 0.0005, 0.05, 10**-16)
print(ans)
print(func_of_i(ans))
print(func_of_i(0.0055507819))